In [43]:
import pandas as pd
import re
from datetime import datetime, timedelta
import os
import pathlib

In [60]:
def convert_file(directory, file):
    rows = []

    with open(directory + file) as f:
        next_line = f.readline()
        while next_line:
            line = next_line

            pattern = re.compile("\d+")


            if re.match(pattern, line):
                line = line.strip()
                try:
                    date = datetime.strptime(line, "%Y %b %d")
                except:
                    Exception("Error parsing date: ", line)
                    exit(1)

            pattern = re.compile("[A-Z]+")

            if "date" in locals() and re.match(pattern, line):
                line = line.strip()

                #Parse with lat / long
                pattern = re.compile("[NS][0-9\-]")
                match = re.search(pattern, line)

                if match:
                    station = line[:match.span()[0]].strip()
                    line = line[match.span()[0]:]

                    lat_pattern = re.compile("[NS][0-9\-\s]{2}")
                    match = re.search(lat_pattern, line)
                    lat = line[match.span()[0]:match.span()[1]].strip()
                    line = line.replace(lat, "")

                    long_pattern = re.compile("[EW][0-9\-\s]{3}")
                    match = re.search(long_pattern, line)
                    long = line[match.span()[0]:match.span()[1]].strip()
                    line = line.replace(long, "")

                    indices = line.split()

                    lat = lat.replace("N", "")
                    lat = lat.replace("S", "-")

                    long = long.replace("E", "")
                    long = long.replace("W", "-")
                    long = long.replace(" ", "")

                    
                #Parse without lat / long
                else:
                    pattern = re.compile("[0-9\-]")
                    match = re.search(pattern, line)
                    station = line[:match.span()[0]].strip()
                    line = line.replace(station, "")

                    indices = line.split()

                a_index = indices[0]

                for i in range(1,len(indices)):
                    row = {
                        'station': station,
                        'geomagnetic_dipole_lat': lat,
                        'geomagnetic_dipole_long': long,
                        'start_time': date + timedelta(hours = 3 * i),
                        'end_time': date + timedelta(hours = 3 * (i + 1)),
                        'A_index': a_index,
                        'K_index': indices[i],
                    }

                    rows.append(row)
                

            next_line = f.readline()

    df = pd.DataFrame(rows, columns = [
        'station',
        'geomagnetic_dipole_lat',
        'geomagnetic_dipole_long',
        'start_time',
        'end_time',
        'A_index',
        'K_index',
        ],
        # dtype=[
        # ('station', "str"),
        # ('geomagnetic_dipole_lat', "int8"),
        # ('geomagnetic_dipole_long', "int8"),
        # ('start_time', "datetime64"),
        # ('end_time', "datetime64"),
        # ('A_index', "int8"),
        # ('K_index', "int8"),
        # ]
    )
    os.makedirs(f"{directory}/prepped/", exist_ok=True)
    df.to_csv(f"{directory}/prepped/{pathlib.Path(file).stem}.csv", index=False)

In [61]:
file = "/Users/dave/code/old-betsy/data/SWPC/AK_values/202201AK.txt"

directory="/Users/dave/code/old-betsy/data/SWPC/AK_values/"

for file in os.listdir(directory):
    if pathlib.Path(file).suffix == ".txt":
        print(directory+file)
        print(pathlib.Path(directory+file).stem)
        convert_file(directory, file)



/Users/dave/code/old-betsy/data/SWPC/AK_values/202201AK.txt
202201AK


In [ ]:
df.dtypes